# Order Brushing - Data Analysis
Shopee code league round 1

### Given Assumption and Definition 
- Brushing shop = concentrate rate greater of equal to 3 at any instances
- Concentrate rate = Number of orders within 1 hour/ Number of unique Buyers within 1 hours
- Brushing buyer = buyer that contributed the ```highest proportion of orders to a shop```
- Highest proportion of orders should include the ordres that occured in instances when brushing

### Basic Concepts
- Each ```orderid``` represnets a distinct transation on Shopee.
- Each unique ```shopid``` is a distinct seller on Shopee.
- Each unique ```userid``` ia a distinct buyer on Shopee.
- Event time refers to the exact time that an order was replaced on Shopee.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('order_brush_order.csv')

In [3]:
df.columns

Index(['orderid', 'shopid', 'userid', 'event_time'], dtype='object')

In [4]:
df.head(5)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [5]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [6]:
df.shape

(222750, 4)

In [7]:
df.isna().sum()

orderid       0
shopid        0
userid        0
event_time    0
dtype: int64

In [8]:
def get_sorted_unique_by_column_name(column_name):
    t = df[column_name].unique()
    t.sort()
    return t

unique_shop_ids = get_sorted_unique_by_column_name('shopid')

In [9]:
groupby_shop_id_df = df.groupby('shopid')

In [10]:
def get_1hour_window(t):
    return t + pd.Timedelta(hours = 1)

In [32]:
def get_time_range_to_brushing_user_ids_to_max_number_of_orders_pair(same_shop_id_df, time_range):
    (lower_time, upper_time) = time_range
    windowed_df = same_shop_id_df[(lower_time <= same_shop_id_df['event_time']) & (same_shop_id_df['event_time'] <= upper_time)]
    
    number_of_orders = len(windowed_df)
    unique_user_ids = windowed_df['userid'].unique()
    
    if float(number_of_orders)/ float(len(unique_user_ids)) >= 3.0:
        user_id_to_number_of_orders_pairs = [
            (user_id, len(windowed_df[windowed_df['userid'] == user_id]))
            
            for user_id in unique_user_ids 
        ]
        
        max_number_of_orders = max([
            number_of_orders
            
            for _, number_of_orders
            in user_id_to_number_of_orders_pairs
        ])
        
        brushing_user_ids = [
            user_id
            
            for user_id, number_of_orders
            in user_id_to_number_of_orders_pairs
            if number_of_orders == max_number_of_orders
        ]
        return (time_range, brushing_user_ids, max_number_of_orders)
    else:
        return (time_range, [], 0)

In [33]:
def flatten(list_of_list):
    return [val for sublist in list_of_list for val in sublist]

In [34]:
def get_brushing_use_id_by_shop_id(shop_id):
    same_shop_id = groupby_shop_id_df.get_group(shop_id)
    
    time_ranges = [
        (event_time, get_1hour_window(event_time))
        for event_time in same_shop_id['event_time']
    ]
    
    time_ranges_to_brushing_use_ids_to_max_number = [
        get_time_range_to_brushing_user_ids_to_max_number_of_orders_pair(
            same_shop_id, time_range) for time_range in time_ranges  
    ]
    
    ultimate_max_number_of_orders = max([
        max_number_of_orders
        
        for _, _, max_number_of_orders
        in time_ranges_to_brushing_use_ids_to_max_number]
    )
    
    ultimate_brushing_user_ids = list(set(flatten([
        brushing_user_ids
        
        for _, brushing_user_ids, max_number_of_orders
        in time_ranges_to_brushing_use_ids_to_max_number
        if max_number_of_orders == ultimate_max_number_of_orders
    ])))
    
    ultimate_brushing_user_ids.sort()
    
    if len(ultimate_brushing_user_ids) > 0:
        stringified_user_ids = [ 
            str(user_id) 
            for user_id in ultimate_brushing_user_ids
        ]
        return '&'.join(stringified_user_ids)
    else:
        return str(0)

In [35]:
shop_id_brushing_use_ids_pairs = [
    (shop_id, get_brushing_use_id_by_shop_id(shop_id))
    for shop_id
    in unique_shop_ids
]

In [38]:
result_df = pd.DataFrame(shop_id_brushing_use_ids_pairs, columns = ['shopid', 'userid'])

In [40]:
result_df.head(5)

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0


In [41]:
result_df[result_df['userid'] == '0'].head(5)

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0


In [43]:
result_df[(result_df['userid'] != '0')].head(5)

,shopid,userid
40,10402,77819
57,10536,672345
111,42472,740844
114,42818,170385453
129,76934,190449497


In [46]:
result_df[result_df['userid'].str.contains('&')].head(5)

,shopid,userid
3189,51134277,29857724&212200633
3341,54257623,1974334&107414154
9045,143281052,99517130&186080843
9844,155143347,156202149&214265994
10024,156883302,76102350&188025647


In [50]:
number_brushing_shop_ids = len(result_df[result_df['userid'] != '0'])
number_non_brushing_shop_ids = len(result_df[~(result_df['userid'] != '0')])

print('Number of brushing shop id =', number_brushing_shop_ids)
print('Number of non brushing shop id =', number_non_brushing_shop_ids)

Number of brushing shop id = 276
Number of non brushing shop id = 18494


In [52]:
brushing_user_with_single_shop = len(result_df[(result_df['userid'] != 
                                              '0') & (result_df['userid'].str.contains('&'))])
brushing_user_with_multiple_shop = len(result_df[(result_df['userid'] != '0') & ~(result_df['userid'].str.contains('&'))])

print('Number of brushing user on single shop =',brushing_user_with_single_shop)
print('Number of brushing use on multiple shop =',brushing_user_with_multiple_shop)

Number of brushing user on single shop = 7
Number of brushing use on multiple shop = 269


In [54]:
result_df

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


In [55]:
result_df.to_csv('submission.csv', index = False)